# 3: Fermipy

There is a Python library named ***fermipy*** that was developed at NASA Goddard by (mainly) Matt Wood with the help of several others. It is very good at simplifying the whole *Fermi* analysis chain - as we will demonstrate shortly. 

The following tutorial should take place inside the included directory called "NGC1275_fermipy". If you're using the Durham tutorial docker image, you'll find all the analysis files inside your locally mounted directory i.e. ```/workdir/tutorial3-fermipy/NGC1275_fermipy```. As we step through the tutorial this directory will be created below and for the sake of this tutorial we'll assume you're using the docker image. If not, for example you're using your own installation of ***fermipy***, you may need to adjust all paths accordingly.

## Intro

Analysis using ***fermipy*** requires a configuration file in the YAML format, which is then used to run the whole analysis chain along with optimized fitting routines and several other bells and whistles to make your analysis **significantly** easier. Firstly let's have a look at the default configuration, which has all of the different possible parameters in it for your perusal.

In [ ]:
!cat config.yaml

Firstly, lets check the current directory...

In [ ]:
current_directory = %pwd
print(current_directory)

The following cell is just for some housekeeping to ensure the analysis works within the Docker image directory structure we've set up. Lets just check the output of the PFILES environment variable...

In [ ]:
import os
# Step 1: Get the current working directory
current_dir = os.getcwd()
print(f" Current directory = {current_dir}")

# Step 2: Get the existing PFILES environment variable (if it exists)
pfiles_dir = os.getenv('PFILES','')
print(f" PFILES env. var. = {pfiles_dir}")

# Step 3: Append the current directory to PFILES
if pfiles_dir:
    # If PFILES is already set, append the current directory
    new_pfiles = f"{current_dir};{pfiles_dir}"
else:
    # If PFILES is not set, just use the current directory
    new_pfiles = current_dir

# Step 4: Update the PFILES environment variable
os.environ['PFILES'] = new_pfiles

# Step 5: Print the new PFILES value for verification
print("Updated PFILES:", os.environ['PFILES'])

Let's list all the files in the current directory...

In [ ]:
!ls -lhtr

Okay, finally we're ready to start analysing using ***fermipy***. Firstly we instantiate the GTAnalysis class into a new object called "gta":

In [ ]:
from fermipy.gtanalysis import GTAnalysis

# Create an analysis object from the config file
gta = GTAnalysis('config.yaml', logging={'verbosity': 3})

We can normally ignore any 'warning' messages, they're generally benign and usually are there to inform you of possible deprecations in future releases for example. Now we'll run our setup.

Note: The next step will take about 10 minutes to complete so you'll need to be patient.

In [ ]:
gta.setup()

We can immediately see how powerful ***fermipy*** is, based on the output (in the red textbox) of this one command. Given the parameters we defined in the config **YAML** file, the setup function has run *gtselect*, and filtered our photon files in the same way that the command line tool would have, by ROI, time, event class and event type. It has then created a livetime cube, using *gtltcube* (actually, it skipped this step, because we already created it for you, to save you some time). It then ran *gtbin* to bin our photons, *gtexpcube2* to compute exposure, and *gtsrcmaps*, to generate a source map of our ROI. 

This is all exceptionally useful, as this allows us to essentially run all of the science tools from Tutorial 2, in one command, based on our input from the configuration file. It saves us time and effort, and reduces the odds of human error! Nice!

Now we've got our model set up, we should try and fit it to our data. The first method we can use is to optimise the fit. 

There are several methods of statistically fitting models, $ \chi ^2$ minimisation and linear regression being two you have probably encountered the most. As our data is multidimensional (in space, time and energy), we use a more suitable approach, as described in [Mattox et al (1996)](https://ui.adsabs.harvard.edu/abs/1996ApJ...461..396M/abstract), called Maximum Likelihood Estimation (MLE). 

Each source in our model has different parameters, with associated statistical likelihoods. By varying the parameters to maximise the likelihoods, we improve the fit of our model. This method is particularly suitable for our data and modelling. You don't need to know in great depth about MLE to do *Fermi* analysis, however it really gives you a greater appreciation of what you're doing mathematically. We'll leave it to you to research this for yourselves.

The first thing we should do with our model is optimise it. Optimisation is a ***fermipy*** routine which iteratively pushes the parameters of each source close to their global maxima, and thus gives us a reasonable fit for our model to the data.

Run the cell below to run the optimisation.

In [ ]:
gta.optimize()

There we go! Our optimization is finished and we should now have a model which represents the data in a much better way than before. But can we do better?

HINT: When conducting ***fermipy*** analysis, you should always run the optimisation step at least once.

As mentioned before, each source has a set of parameters e.g. normalisation of the spectral power law, and spectral shape etc. By freeing these parameters, we can execute a fit with respect to these. A fit provides a much better, well, fit, of the model to the data with respect to the freed parameters, however is very time consuming for our machines to do. In the case of NGC 1275 (which we are looking at) we probably want to fit the normalisation, and the normalisation of the sources around it. Parameters like spectral shape can be left alone, because we are pretty convinced that it is a point source, and there is no evidence for extended $\gamma$-ray emission.

First, we free the normalisation for a 3 degree region centered on NGC 1275, and also our background components in the model:

In [ ]:
gta.free_sources(distance=3.0,pars='norm') #frees the point sources
gta.free_source('galdiff', pars='norm') #frees the galactic diffuse
gta.free_source('isodiff', pars='norm') #frees the isotropic diffuse

Now we have defined the sources and which of their parameters are free to vary, we can execute our fit with this simple command:

In [ ]:
gta.fit()

...and we're done!

Using ***fermipy*** we've successfully built a model using the Science Tools, and fitted it to our data in a few simple lines of code. Now we've done our modelling, we should really check how accurate it is. Fortunately, there is an easy way to do this, called a residual map. Our data shows the number of photons in each bin, and our model tries to predict this, so by subtracting one from the other on a bin by bin basis, we are able to see how well our model predicts the data.

In [ ]:
gta.residmap(make_plots=True)

As we included the argument ```make_plots=True``` in our command, ***fermipy*** has automatically generated some plots of the residual maps for us to look at. Generally these 'diagnostic plots' are of poor quality and we wouldn't consider them for publication. While the diagnostic plots are extremely useful, these can sometimes (for example SED)be wrong, and the important thing to remember is that these are diagnostic plots and should always be treated as such! To assess the quality of our model let us inspect our residual maps:

First we can look at our data itself:

![data_counts](../img/data_counts.png)

The big bright blob in the centre is our galaxy, NGC 1275 (or 4FGL J0319.8+4130 in the 4th *Fermi*-LAT point source catalog). We can see there are some other, less bright sources nearby, and a large amount of diffuse emission in the top left of the map, this is from the Galactic plane! Clearly though NGC 1275 is the brightest thing in our ROI, which is useful (although not always the case depending on what you're looking at). 

Now though, we can look at our model:

![model_counts](../img/model_counts.png)

The two maps are almost indistinguishable from one another, which is good! We want our modelling to be as representative of our data as possible. Now subtracting the two... 

![rmap_counts](../img/rmap_counts.png)

We can see that across the ROI, we have an excess of counts (the red), which indicates that we are undermodelling the data (i.e. not predicting enough counts). But right in the centre, we have the opposite problem of overmodelling. To see how much of an issue this is, we need to turn these number of counts into something more statistically meaningful, and we do this representing the residuals in significance space:

![rmap_sigma](../img/rmap_sigma.png)

Now we see that our overmodelling and undermodelling are not very statistically significant, and thus we can be confident that our ROI is modelled well.

HINT: Due to the complexity of modelling the diffuse $\gamma$-ray background, statistical fluctuations of $2-3\sigma$ are not uncommon, and hence why the accepted detection threshold for the discovery of a new $\gamma$-ray source is $5 \sigma$

Examining our maps, we should notice that all of our sources in the model are from the 4FGL. This is because when we built the model, ***fermipy*** used the 4FGL to automatically insert all of the known 4FGL sources into our model. It would however be naive to assume that this is the most accurate representation of our ROI. One major issue is sources that may not be catalogued. Luckily, we have a method to deal with that. First, we will generate a TS map of our ROI. The TS (test statistic) is a statistic which arises from our likelihood modelling, but directly correlates to a significance value for $k$ degrees of freedom between hypotheses. For one degree of freedom however, the square root of the TS gives us our significance value, so a TS of 9 is $3 \sigma$ etc. When we generate a TS map, it will give us the significance of any unmodelled emission across the ROI, and thus let us see if there is any emission left to be modelled!

In [ ]:
gta.tsmap(make_plots=True)

If we now open and inspect our generated TS map (in significance) we can see there are a few hot spots of emission, but nothing too much to worry about. 

![TS_sigma](../img/TS_sigma.png)

To be safe, we can run a source finding algorithm, which will fit a point source to any position with a significance $> 4 \sigma$. It does like to take a few minutes though, so use this opportunity to go and get a coffee/beer depending on whether its before/after 4pm.

In [ ]:
gta.find_sources(sqrt_ts_threshold=4.0, min_separation=0.5)

And we're done! If we now generate a new TS map, we'll see..

In [ ]:
gta.tsmap(make_plots=True)

... we now have some more sources added! That is very handy and should account for some of the errant emission in our ROI. 

![TS_sigma2](../img/TS_sigma2.png)

Now we are absolutely 100% certain our ROI is as good as it can be (to be honest, we might have gone slightly overboard, but we wanted to show you all the features of ***fermipy***). Anyway, now that we have got this far it would be extremely annoying if we were to lose all of this work! So now is a good time to tidy up our ROI and save it. When we say tidy up, we mean deleting all of our low significance sources. Just because there is a catalogued source in the 4FGL, this doesn't mean it is significant in our ROI, so we delete anything of significance $ < 3 \sigma$

In [ ]:
import numpy as np
gta.delete_sources(minmax_ts=[-np.inf, 9])

And now we will save our ROI, and generate some plots while we do so:

In [ ]:
gta.write_roi('NGC_1275_fit', make_plots=True)

We can load an ROI back in with ```gta.load('NGC_1275_fit')``` if we need to, after defining gta again.

You can feel free to peruse these plots at your own leisure, but for now, we will move on to some more advanced features of ***fermipy*** for source analysis.

The first thing we normally want to do, is to calculate a spectral energy distribution (SED) of the source. This shows the relationship between the flux and the energy of the source. Thankfully ***fermipy*** can do this for us quite easily. 

In [ ]:
sed = gta.sed('4FGL J0319.8+4130') 

As mentioned before, the diagnostic plots for SEDs are often flat out incorrect, which means we need to plot our own. So lets do that!

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline


ts = sed['ts']
dnde = sed['dnde']
flux_error = sed['e2dnde_err']
e_min = sed['e_min']
e_ref = sed['e_ref']
e_max = sed['e_max']
uplim = sed['e2dnde_ul95']

flux = (e_ref**2)*dnde
yval = []
ts_int = []
uplim_bool = []
i = 0
i_array = []


while i < len(ts):
    ts_int.append(int(ts[i]))
    if ts_int[i] < 4:
        yval.append(uplim[i])
        uplim_bool.append(1)
        
    else:
        yval.append(flux[i])
        uplim_bool.append(0)
        i_array.append(i)
    i = i + 1

uplims = np.array(uplim_bool, dtype=bool)

low_xerr = []
high_xerr = []
j = 0

flux_valid = []
E_ctr_valid = []
for m in i_array:
    flux_valid.append(flux[m])
    E_ctr_valid.append(e_ref[m])


while j < len(e_ref):
    a = e_ref[j] - e_min[j]
    b = e_max[j] - e_ref[j]
    low_xerr.append(a)
    high_xerr.append(b)
    j = j + 1

xerr = [low_xerr, high_xerr]

fig = plt.subplots(figsize=(14, 8))
plt.errorbar(e_ref, uplim, xerr = xerr, yerr = flux_error
             , uplims = uplims, linestyle = 'None',
             color = 'black', marker = 'o')


plt.title('NGC 1275 Spectral Energy Distribution')
plt.xlabel('Energy [MeV]')
plt.ylabel('Flux MeV cm^(-2) s^(-1)')
plt.xscale('log')
plt.yscale('log')
plt.savefig('sed_NGC_1275.png', dpi = 400)

Now we've plotted our SED, lets take a look (it may or may not have shown up in the output of the above box). 

![SED](../img/sed_NGC_1275.png)

That looks great! our code has plotted our SED, with error bars and energy bin widths, and plotted 95% confidence upper limits on bins where $ \sigma < 2$. We could, of course plot these points too, but low significance bins have large error bars, and we aren't certain enough in them to take them seriously in any fit. We haven't fitted a line to our data, although if we wanted to, we could use $\chi ^2$ minimisation to fit a log parabola to it. 

You are more than welcome to plagiarise this code for plotting, although it is actually pretty old and inelegant (it was written for an old analysis of the Crab Nebula). However, it does work and thats what matters!

Now we know that our SED has a log parabolic shape, and we can see the distribution of flux with energy, what else can we do? 

So far we've been testing NGC 1275 as if it were a point source, however extension in *Fermi* sources does exist (although it is uncommon). Two radio galaxies are known to have extension: Centaurus A (our closest AGN neighbour) and Fornax A. Luckily ***fermipy*** has a lovely routine called 'extension' to test whether a source is a point source, or is extended. Lets run it now and see what happens.

In [ ]:
ext = gta.extension('4FGL J0319.8+4130')

Now we've computed the extension of the source, lets take a look.

In [ ]:
ext['ts_ext']

There is a great deal of information in the numpy array produced by the extension algorithm. Luckily however, we are given a test statistic of extension. This test statistic gives us an indication as to how extended a source is. Remember, the square root of the TS is the significance! Thus our threshold for claiming extension is $TS = 25$ or $5 \sigma$. Our TS of extension is 0.05(ish) is far below this, and thus our hypothesis that NGC 1275 is a point source, remains correct. 

We can also look at the other parameters from the extension fit, but this is rather pointless considering that the source isn't extended! We'll leave it to your curiosity to explore. 

There are two other useful tools we'll explore with ***fermipy*** for point source analysis. The first is localisation, which iteratively tests the position of our source in a whole load of different places, thus shifting our source position to the most likely place in the model. When we add our sources to the model from the catalogue, it takes the RA and Dec from the catalogued position. However, this might not always be the most accurate. In particular, we can use localisation to improve the positional accuracy of sources added with the find sources algorithm. Lets give it a go.

In [ ]:
loc = gta.localize('4FGL J0319.8+4130')

And we're done. Lets take a look inside the numpy array to see our new position.

In [ ]:
loc['ra'], loc['dec']

These are our best fit coordinates for NGC 1275, given our year of data. We can also see that they really aren't dramatically different to what they originally were. In fact, it would be very surprising and slightly worrying if they were!

Now on to the final key bit of point source analysis: the light-curve. Light-curves are one of the most useful, and powerful tools in an astronomers arsenal, an easy way to probe whether the flux of a source varies in time. Unfortunately, in ***fermipy***, they take a long time to make, due to the fact we need to compute the livetime in each bin. The command to produce a light-curve is slightly more complex than just putting in the name of our source, so we will go through these together now. 

First, we have the name of our source (obviously). Then we specify the number of time bins for our light-curve using the ```nbins``` parameter. As we have 1 year of data, we can look at the flux in monthly bins. This is a sensible timescale, and with most objects it is difficult to see anything at all with smaller bins. The ```multithread``` option allows our lightcurve algorithm to split up its jobs between the separate cores on our computer, thus allowing us to simultaneously compute several bins at once. This has the effect of speeding up our lightcurve by a factor of 2-4, with the side effect of essentially bricking our computer until its finished. We can choose to use scaled source maps by setting the ```use_scaled_srcmap``` parameter to true. This allows us to gain a huge advantage in computing speeds, by sacrificing a little accuracy in our modelling. This loss of accuracy is normally negligible, especially for an off the plane source like NGC 1275. The ```save_bin_data``` parameter allows the saving of bin data in individual folders, which leads to more reliable lightcurve generation when multithreading is on, and we highly recommend you enable this.

Lets run it and see what happens.

In [ ]:
lc = gta.lightcurve('4FGL J0319.8+4130', nbins=12, multithread=True,
                    use_scaled_srcmap=True, save_bin_data=True)

gta.lightcurve will not produce diagnostic plots, so we need to plot these ourselves.

In [ ]:
import numpy as np
lc=np.load('4fgl_j0319.8+4130_lightcurve.npy',allow_pickle=True)

In [ ]:
import matplotlib.pyplot as plt
ts_1 = lc[()]['ts']
tmin_1 = lc[()]['tmin_mjd']
tmax_1 = lc[()]['tmax_mjd']
flux_1 = lc[()]['eflux']
ul_1 = lc[()]['eflux_ul95']
flux_err_1 = lc[()]['eflux_err']

tcen = []
i = 0

ts = []
tmin = []
tmax = []
flux = []
ul = []
flux_err = []
i = 0
while i < len(ts_1):
    ts.append(ts_1[i])
    tmin.append(tmin_1[i])
    tmax.append(tmax_1[i])
    flux.append(flux_1[i])
    ul.append(ul_1[i])
    flux_err.append(flux_err_1[i])
    i = i + 1


where_are_NaNs = np.isnan(ts)
i = 0
while i < len(where_are_NaNs):
    if where_are_NaNs[i]==True:
        ts[i] = 0
        tmin[i] = 0
        tmax[i] = 0
        flux[i] = 0
        ul[i] = 0
        flux_err[i] = 0
        i = i + 1
    else:
        i = i + 1

i = 0
while i < len(ts):
    if ts[i] == 0:
        ts.remove(ts[i])
        tmin.remove(tmin[i])
        tmax.remove(tmax[i])
        flux.remove(flux[i])
        ul.remove(ul[i])
        flux_err.remove(flux_err[i])
        i = i - 1
    i = i + 1

i = 0
while i < len(tmin):
    t = (tmin[i] + tmax[i])/2
    tcen.append(t)
    i = i + 1

flux_plot = []
flux_err_plot = []
ts_int = []
uplim_bool = []
j = 0

while j < len(ts):
    ts_int.append(int(ts[j]))
    
    if ts_int[j] < 4:                ### CHANGE TS THRESHOLD HERE
        flux_plot.append(ul[j])
        uplim_bool.append(1)
        flux_err_plot.append(2e-6)
        
    else:
        flux_plot.append(flux[j])
        uplim_bool.append(0)
        flux_err_plot.append(flux_err[j])
    j = j + 1
    
uplims = np.array(uplim_bool, dtype = bool)

low_xerr = []
k = 0

while k < len(tcen):
    a = tcen[k] - tmin[k]
    low_xerr.append(a)
    k = k + 1
high_xerr = low_xerr
    
xerr = [low_xerr, high_xerr]

fig = plt.subplots(figsize=(14, 8))
plt.errorbar(tcen, flux_plot, xerr = xerr, yerr = flux_err_plot,
             uplims = uplims, linestyle = 'None', color = 'k',
             marker = 'None')
plt.title('NGC 1275 Light-Curve')
plt.xlabel('Time [MJD]')
plt.ylabel('Flux MeV cm^(-2) s^(-1)')
plt.yscale('log')
plt.xlabel("Time (MJD)")
plt.ylabel("Flux ($MeV cm^{-2} s^{-1}$)")
plt.savefig('NGC_1275_lc', dpi=400)

Lets have a look!

![lc](../img/NGC_1275_lc.png)

By eye, that looks variable! NGC 1275 is a relatively bright $\gamma$-ray source, and is known for exhibiting variability. During past states of high gamma-ray activity it's been possible to achieve 12-hour bin sizes, which is quite impressive for an object of this nature. Again, we can fit a curve to this data, but for now we are done!

Congratulations, you have just executed a more or less complete analysis of a radio galaxy using NASA's *Fermi* $\gamma$-ray space telescope! 

![welldone](../img/welldone.jpg)

If you are ready for more of a challenge, then you can look at the Crab Nebula. The Crab Nebula consists of a pulsar wind nebula (4FGL J0534.5+2201e) and its central pulsar both are spatially coincident with each other, however the Crab is on the galactic plane, so this is more of a challenge to model. 

To look at the Crab, you will need to download a dataset for it (following the instructions in tutorial 1) and then write a **YAML** configuration file (you can copy and change the NGC 1275 one) and execute an analysis chain using the ***fermipy*** library. We recommend setting up a separate directory to do this in. The methods should be largely the same. We've studied the Crab as a project before, so if you have any issues, please let us know and we can try and help. Good Luck!